In [13]:
import mvpa2
from os.path import exists, join  
import mvpa2.suite as mv
import glob
import numpy as np
from scipy.stats import zscore

In [2]:
# set everything up ! 
subid = [1,12,17,27,32,33,34,36,37,41]
subjs = ['{:0>6}'.format(i) for i in subid]
hemis = ['lh', 'rh']
data_path = '/dartfs-hpc/scratch/psyc164/mvpaces/glm/'
datalad_path = '/dartfs/rc/lab/D/DBIC/DBIC/datasets.datalad.org/labs/haxby/attention/derivatives/'


# change accordingly
tas ks = ['beh','tax']
sub, hemi = subjs[0], hemis[0]
prefix = 'sub-rid'+sub
suffix = hemi+'.coefs.gii'


taxonomy = np.repeat(['bird', 'insect', 'primate', 'reptile', 'unduate'],4)
behavior = np.tile(['eating', 'fighting', 'running', 'swimming'],5)
conditions = [' '.join((beh, tax)) for beh, tax in zip(behavior, taxonomy)]

In [3]:
# grab all subj1's  files into a list
#fn = data_path+sub_pref+'*lh.coefs.gii'
fn = data_path+prefix+'*'+suffix
files = sorted(glob.glob(fn))

In [23]:
targets = range(1,21)
ds = None
for x in range(len(files)):
    chunks = [x+1]*20
    d = mv.gifti_dataset(files[x], chunks=chunks, targets=targets)
    d.sa['conditions']=conditions
    d.sa['taxonomy'] = taxonomy
    d.sa['behavior'] = behavior
    if ds is None:
        ds = d
    else:      
        ds = mv.vstack((ds,d))

In [24]:
ds.fa['node_indices']=range(ds.shape[1])
train='behavior'

In [25]:
ds.samples = zscore(ds.samples, axis=1)

In [26]:
# load surface and get searchlight query
radius = 3 
surface = mv.surf.read(join(data_path, '{0}.pial.gii'.format(hemi)))
# this is an arbitrary radius and distance metric!
query = mv.SurfaceQueryEngine(surface, radius, distance_metric='dijkstra')
# based off PyMVPA tutorial
clf = mv.LinearCSVMC()
cv = mv.CrossValidation(clf, mv.NFoldPartitioner(attr=train),
                     errorfx=lambda p, t: np.mean(p == t),
                     enable_ca=['stats'])
searchlights = mv.Searchlight(cv, queryengine=query, postproc=mv.mean_sample(), roi_ids=None)

In [ ]:
sl_clf_results = searchlights(ds)